In [ ]:
# Imports
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)

from src import *
import scipy.sparse.linalg as spsl
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# Algorithm Parameters
max_energy_level = 4
kd_ratio = 2.5
noise_threshold_values = 1/(10**np.arange(0,5,.25))
epsilon = 1e-3
K = 500
delta_t = 0.39
num_trials = 20

# Hamiltonian Parameters
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.595'
basis_set = 'sto-3g'
mapper = 'parity'

In [ ]:
# Generate Hamiltonian and get true eigenenergies
hamiltonian = molecular_hamiltonian(molecule,basis_set,mapper)
sparse_hamiltonian = hamiltonian.to_matrix(sparse = True)
num_qubits = hamiltonian.num_qubits

v, w = spsl.eigsh(sparse_hamiltonian,k=20+1,which = 'SA')
true_eigenenergies = np.unique(np.round(v,8))[:max_energy_level+1]

In [ ]:
# Construct reference state
indices = np.argsort(sparse_hamiltonian.diagonal())
reference_state = bitstring_superposition_state(num_qubits,[bin(indices[i])[2:] for i in [0,1,1,2,3,4,5]])

# Get evolved reference states
max_K = K
max_d = int(max_K/kd_ratio)
time_evolution_operator = -1j*sparse_hamiltonian*delta_t
evolved_reference_states = spsl.expm_multiply(time_evolution_operator,reference_state,start=0,stop=max_d+max_K+1,num = max_d+max_K+2)

In [ ]:
# Get MODMD observables
pauli_strings, coeffs = zip(*sorted(hamiltonian.to_list(), key = lambda x: np.abs(np.real(x[1])), reverse = True))

modmd_observables = ([SparsePauliOp('I' * num_qubits).to_matrix(sparse=True)] + 
                     [SparsePauliOp(p).to_matrix(sparse=True) for p in pauli_strings[54:66]])

In [ ]:
# MODMD Results
modmd_results = []

I_values = list(range(1,len(modmd_observables)+1))
X_elements = generate_X_elements(modmd_observables,max_d,max_K,reference_state,evolved_reference_states)

for noise_threshold in noise_threshold_values:

    noise_threshold_results = []

    for trial in range(num_trials):

        gaussian_noise = np.random.normal(0,epsilon,X_elements.shape) + 1j * np.random.normal(0,epsilon,size=X_elements.shape)
        noisy_X_elements = X_elements + gaussian_noise
        
        noise_threshold_results.append(varying_I_results(I_values,noise_threshold,noisy_X_elements,delta_t,K,kd_ratio,max_energy_level))
  
    modmd_results.append(noise_threshold_results) 

In [ ]:
# Compute errors
absolute_modmd_errors = np.array([np.abs(np.array(modmd_results)[:,i,:,:] - true_eigenenergies) for i in range(num_trials)])

In [ ]:
# Plotting
fig, axes = plt.subplots(1,4, figsize = (12,10), constrained_layout=True,sharex=True)
matplotlib.rcParams.update({'font.size': 14})
extent = (I_values[0], I_values[-1], np.log10(noise_threshold_values[-1]), np.log10(noise_threshold_values[0]))

for energy_level, ax in enumerate(axes.flat):

    modmd_average = np.average(absolute_modmd_errors,0)[:,:,energy_level]
    im = ax.imshow(np.log10(modmd_average), cmap = 'RdPu_r', extent=extent,vmin=-6.9,vmax=-.1)

    ax.set_aspect(2.8)
    ax.set_title(f'Absolute Error in $E_{energy_level}$')
    if energy_level != 0:
        ax.set_yticks([])
    ax.axhline(np.log10(epsilon),color = '#95ab9b', linestyle = '--',)

axes[0].set_ylabel(r'$\log_{10}(\tilde\delta)$')
fig.supxlabel('# of observables',y=.28)
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([1.02, 0.345, 0.03, 0.31])
fig.colorbar(im, cax=cbar_ax)